In [2]:
from __future__ import print_function

import numpy as np
import warnings

from keras.layers import merge, Input, Lambda, Layer
from keras.layers import Dense, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model, Sequential
from keras.preprocessing import image
import keras.backend as K
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file



Using TensorFlow backend.


In [ ]:
kvar = K.random_uniform_variable((1,1), 0, 1)[0][0]
K.eval(kvar)

In [ ]:
input_layer = Input(shape=(5,))
x = Dense(32)(input_layer)
x = Dense(3)(x)
y = (Lambda(lambda x:x*np.random.random()))(x)

In [ ]:
input_layer = Input(shape=(5,))
x = Dense(32)(input_layer)
x = Dense(3)(x)
r = K.random_uniform_variable((1,1), 0, 1)[0][0]
y = (Lambda(lambda x:x*r ))(x)

In [24]:
input_layer = Input(shape=(5,))
x1 = Dense(32)(input_layer)
x1 = Dense(3)(x1)
x2 = Dense(32)(input_layer)
x2 = Dense(3)(x2)
alpha = K.random_normal(K.shape(x1),0,1)
y1 = (Lambda(lambda x:x*alpha ))(x1)
y2 = (Lambda(lambda x:x*alpha ))(x2)
y = merge([y1,y2], mode='sum')

In [32]:
K.shape(x1)

<tf.Tensor 'Shape_3:0' shape=(2,) dtype=int32>

In [25]:
model1 = Model(input_layer, y)

In [22]:
model2 = Model(input_layer, y)

In [26]:
X = np.random.random((4,5))

In [29]:
Y1 = model1.predict(X)

In [25]:
Y2 = model2.predict(X)

In [30]:
Y1

array([[ 0.17488301, -0.32657152, -0.45682985],
       [-0.00968019, -0.85604554, -0.75447154],
       [-0.06756634, -0.32657766, -0.27654845],
       [-0.00598352,  1.51511335,  0.08383514]], dtype=float32)

In [19]:
class RandomAffine(Layer):

    def __init__(self, **kwargs):
        super(RandomAffine, self).__init__(**kwargs)

    def call(self, inputs, mask=None):
        x,y = inputs[0],inputs[1]
        alpha = K.random_normal(K.shape(x),0,1)
        return alpha * x + (1-alpha) * y


In [ ]:
kvar = K.random_uniform_variable((1,1), 0, 1)[0][0]
K.eval(kvar)